# Imports

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import matplotlib.pyplot as plt # plotting
import matplotlib.image as mpimg # images
import numpy as np #numpy
import seaborn as sns
import tensorflow.compat.v2 as tf #use tensorflow v2 as a main 
import tensorflow.keras as keras # required for high level applications
from sklearn.model_selection import train_test_split # split for validation sets
from sklearn.metrics import accuracy_score, f1_score, classification_report
import scipy
import pandas as pd
import re

In [ ]:
def cleanTexts(texts):
    cleaned = []
    pattern = "[^a-zA-ZÀ-ÿ0-9]"
    for text in texts:
        clrd = re.sub(pattern," ",text).lower().strip()
        cleaned.append(clrd)
    return cleaned

# Load data

In [ ]:
from sklearn.utils import shuffle

train_data = pd.read_csv('train_data_tweetsFR.csv')
train_data = shuffle(train_data)
train_data.head()

,text,label
884011,Avez-vous déjà sorti votre internet pendant qu...,0
40240,Jusqu'à 4:39 am en faisant du riz pour mon chi...,0
162598,Oui tu as raison. Chaque relation a de la plac...,1
599169,Je pense que j'ai des vertiges ...,0
769434,"Omg, j'espère que ce n'est pas la grippe. Va b...",0


In [ ]:
x_train = list(train_data['text'])
# print(x[:5])

y_train = list(train_data['label'])
# print(y[:5])

#x_train = x_train[:20000]
#y_train = y_train[:20000]

#x_train = x_train[:500000]
#y_train = y_train[:500000]

x_train = x_train[:900000]
y_train = y_train[:900000]

In [ ]:
test_data = pd.read_csv('test_data_tweetsFR.csv')

x_test = list(test_data['text'])
y_test = list(test_data['label'])

# Train model and predict on test dataset

In [ ]:
from tensorflow import string as tf_string
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.compat.v1.keras.layers import CuDNNGRU, CuDNNLSTM
from tensorflow.keras.layers import LSTM, GRU, Bidirectional

# 1 - 6 Vectorizer parameters

In [ ]:
embedding_dim = 128 # Dimension of embedded representation 
vocab_size = 30000 # Number of unique tokens in vocabulary
sequence_length = 30 # Output dimension after vectorizing

vect_layer = TextVectorization(max_tokens=vocab_size, output_mode='int', output_sequence_length=sequence_length)
vect_layer.adapt(x_train)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=69, stratify=y_train)

## 1.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_vect = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim)(x_vect)
x = LSTM(64, activation='relu', return_sequences=True)(emb)
x = GRU(64, activation='relu', return_sequences=True)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 30)               0         
 torization)                                                     
                                                                 
 embedding_6 (Embedding)     (None, 30, 128)           3840000   
                                                                 
 lstm_6 (LSTM)               (None, 30, 64)            49408     
                                                                 
 gru_6 (GRU)                 (None, 30, 64)            24960     
                                                                 
 flatten_6 (Flatten)         (None, 1920)              0         
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 256
epochs = 2
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/2
71/71 [==============================] - 26s 319ms/step - loss: 0.6317 - accuracy: 0.6260 - val_loss: 0.5472 - val_accuracy: 0.7165
Epoch 2/2
71/71 [==============================] - 17s 234ms/step - loss: 0.4395 - accuracy: 0.8038 - val_loss: 0.5302 - val_accuracy: 0.7390


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 6s 18ms/step - loss: 0.5122 - accuracy: 0.7544


[0.5122210383415222, 0.7544000148773193]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 75.44% 
f1-score is 0.7593100744805957% 
              precision    recall  f1-score   support

           0       0.77      0.73      0.75      5000
           1       0.74      0.77      0.76      5000

    accuracy                           0.75     10000
   macro avg       0.75      0.75      0.75     10000
weighted avg       0.75      0.75      0.75     10000



## 2.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_vect = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim)(x_vect)
x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(emb)
x = Bidirectional(GRU(64, activation='relu', return_sequences=True))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 30)               0         
 torization)                                                     
                                                                 
 embedding_7 (Embedding)     (None, 30, 128)           3840000   
                                                                 
 bidirectional_2 (Bidirectio  (None, 30, 128)          98816     
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 30, 128)          74496     
 nal)                                                            
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 256
epochs = 2
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/2
71/71 [==============================] - 38s 459ms/step - loss: 0.6321 - accuracy: 0.6322 - val_loss: 0.5480 - val_accuracy: 0.7305
Epoch 2/2
71/71 [==============================] - 37s 527ms/step - loss: 0.4274 - accuracy: 0.8104 - val_loss: 0.5620 - val_accuracy: 0.7280


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 17s 54ms/step - loss: 0.5230 - accuracy: 0.7456


[0.522959291934967, 0.7455999851226807]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 74.56% 
f1-score is 0.755666538609297% 
              precision    recall  f1-score   support

           0       0.77      0.70      0.73      5000
           1       0.73      0.79      0.76      5000

    accuracy                           0.75     10000
   macro avg       0.75      0.75      0.75     10000
weighted avg       0.75      0.75      0.75     10000



## 3.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_vect = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim)(x_vect)
x = LSTM(64, activation='relu', return_sequences=True)(emb)
x = GRU(64, activation='relu', return_sequences=True)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 30)               0         
 ectorization)                                                   
                                                                 
 embedding_10 (Embedding)    (None, 30, 128)           3840000   
                                                                 
 lstm_10 (LSTM)              (None, 30, 64)            49408     
                                                                 
 gru_10 (GRU)                (None, 30, 64)            24960     
                                                                 
 flatten_10 (Flatten)        (None, 1920)              0         
                                                          

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 2
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/2
586/586 [==============================] - 158s 263ms/step - loss: 0.4741 - accuracy: 0.7725 - val_loss: 0.4311 - val_accuracy: 0.7996
Epoch 2/2
586/586 [==============================] - 145s 248ms/step - loss: 0.4075 - accuracy: 0.8152 - val_loss: 0.4285 - val_accuracy: 0.8032


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 6s 18ms/step - loss: 0.4161 - accuracy: 0.8091


[0.41607505083084106, 0.8090999722480774]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 80.91000000000001% 
f1-score is 0.8048256824455577% 
              precision    recall  f1-score   support

           0       0.80      0.83      0.81      5000
           1       0.82      0.79      0.80      5000

    accuracy                           0.81     10000
   macro avg       0.81      0.81      0.81     10000
weighted avg       0.81      0.81      0.81     10000



## 4.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_vect = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim)(x_vect)
x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(emb)
x = Bidirectional(GRU(64, activation='relu', return_sequences=True))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 30)               0         
 ectorization)                                                   
                                                                 
 embedding_11 (Embedding)    (None, 30, 128)           3840000   
                                                                 
 bidirectional_4 (Bidirectio  (None, 30, 128)          98816     
 nal)                                                            
                                                                 
 bidirectional_5 (Bidirectio  (None, 30, 128)          74496     
 nal)                                                            
                                                          

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 2
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/2
586/586 [==============================] - 286s 478ms/step - loss: 0.4741 - accuracy: 0.7726 - val_loss: 0.4284 - val_accuracy: 0.7991
Epoch 2/2
586/586 [==============================] - 278s 475ms/step - loss: 0.4038 - accuracy: 0.8173 - val_loss: 0.4239 - val_accuracy: 0.8039


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 10s 30ms/step - loss: 0.4130 - accuracy: 0.8077


[0.41295263171195984, 0.807699978351593]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 80.77% 
f1-score is 0.8055415107695418% 
              precision    recall  f1-score   support

           0       0.80      0.82      0.81      5000
           1       0.81      0.80      0.81      5000

    accuracy                           0.81     10000
   macro avg       0.81      0.81      0.81     10000
weighted avg       0.81      0.81      0.81     10000



## 5.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_vect = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim)(x_vect)
x = LSTM(64, activation='relu', return_sequences=True)(emb)
x = GRU(64, activation='relu', return_sequences=True)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 30)               0         
 ectorization)                                                   
                                                                 
 embedding_12 (Embedding)    (None, 30, 128)           3840000   
                                                                 
 lstm_12 (LSTM)              (None, 30, 64)            49408     
                                                                 
 gru_12 (GRU)                (None, 30, 64)            24960     
                                                                 
 flatten_12 (Flatten)        (None, 1920)              0         
                                                          

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 2
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/2
1055/1055 [==============================] - 274s 257ms/step - loss: 0.4573 - accuracy: 0.7841 - val_loss: 0.4190 - val_accuracy: 0.8078
Epoch 2/2
1055/1055 [==============================] - 269s 255ms/step - loss: 0.4033 - accuracy: 0.8170 - val_loss: 0.4131 - val_accuracy: 0.8109


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 6s 18ms/step - loss: 0.4053 - accuracy: 0.8109


[0.40534713864326477, 0.8108999729156494]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 81.08999999999999% 
f1-score is 0.8079618157814562% 
              precision    recall  f1-score   support

           0       0.80      0.83      0.81      5000
           1       0.82      0.80      0.81      5000

    accuracy                           0.81     10000
   macro avg       0.81      0.81      0.81     10000
weighted avg       0.81      0.81      0.81     10000



## 6.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_vect = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim)(x_vect)
x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(emb)
x = Bidirectional(GRU(64, activation='relu', return_sequences=True))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 30)               0         
 ectorization)                                                   
                                                                 
 embedding_13 (Embedding)    (None, 30, 128)           3840000   
                                                                 
 bidirectional_6 (Bidirectio  (None, 30, 128)          98816     
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 30, 128)          74496     
 nal)                                                            
                                                          

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 2
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/2
1055/1055 [==============================] - 516s 484ms/step - loss: 0.4520 - accuracy: 0.7871 - val_loss: 0.4159 - val_accuracy: 0.8087
Epoch 2/2
1055/1055 [==============================] - 510s 483ms/step - loss: 0.3973 - accuracy: 0.8204 - val_loss: 0.4097 - val_accuracy: 0.8127


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 10s 30ms/step - loss: 0.4002 - accuracy: 0.8127


[0.4002379775047302, 0.8126999735832214]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 81.27% 
f1-score is 0.8153770330211928% 
              precision    recall  f1-score   support

           0       0.82      0.80      0.81      5000
           1       0.80      0.83      0.82      5000

    accuracy                           0.81     10000
   macro avg       0.81      0.81      0.81     10000
weighted avg       0.81      0.81      0.81     10000



# 7 - 8 Vectorizer parameters

In [ ]:
embedding_dim = 128 # Dimension of embedded representation 
vocab_size = 100000 # Number of unique tokens in vocabulary
sequence_length = 30 # Output dimension after vectorizing

vect_layer = TextVectorization(max_tokens=vocab_size, output_mode='int', output_sequence_length=sequence_length)
vect_layer.adapt(x_train)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=69, stratify=y_train)

## 7.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_vect = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim)(x_vect)
x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(emb)
x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(emb)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 30)               0         
 ectorization)                                                   
                                                                 
 embedding_15 (Embedding)    (None, 30, 128)           12800000  
                                                                 
 bidirectional_11 (Bidirecti  (None, 30, 128)          98816     
 onal)                                                           
                                                                 
 flatten_15 (Flatten)        (None, 3840)              0         
                                                                 
 dense_45 (Dense)            (None, 64)                245

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 2
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/2
1055/1055 [==============================] - 256s 240ms/step - loss: 0.4525 - accuracy: 0.7866 - val_loss: 0.4211 - val_accuracy: 0.8052
Epoch 2/2
1055/1055 [==============================] - 249s 236ms/step - loss: 0.3854 - accuracy: 0.8273 - val_loss: 0.4160 - val_accuracy: 0.8095


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 5s 16ms/step - loss: 0.4064 - accuracy: 0.8119


[0.40641912817955017, 0.8119000196456909]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 81.19% 
f1-score is 0.8105167724388033% 
              precision    recall  f1-score   support

           0       0.81      0.82      0.81      5000
           1       0.82      0.80      0.81      5000

    accuracy                           0.81     10000
   macro avg       0.81      0.81      0.81     10000
weighted avg       0.81      0.81      0.81     10000



## 8.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim)(x_v)
x = Bidirectional(LSTM(128, activation='relu', return_sequences=True))(emb)
x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(x)
x = keras.layers.Dropout(0.4)(x)
x = Bidirectional(GRU(64, activation='relu', return_sequences=True))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.4)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 30)               0         
 ectorization)                                                   
                                                                 
 embedding_16 (Embedding)    (None, 30, 128)           12800000  
                                                                 
 bidirectional_12 (Bidirecti  (None, 30, 256)          263168    
 onal)                                                           
                                                                 
 bidirectional_13 (Bidirecti  (None, 30, 128)          164352    
 onal)                                                           
                                                          

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 768
epochs = 2
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/2
1055/1055 [==============================] - 721s 676ms/step - loss: 0.4602 - accuracy: 0.7862 - val_loss: 0.4150 - val_accuracy: 0.8098
Epoch 2/2
1055/1055 [==============================] - 704s 667ms/step - loss: 0.3913 - accuracy: 0.8280 - val_loss: 0.4162 - val_accuracy: 0.8103


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 13s 40ms/step - loss: 0.4066 - accuracy: 0.8103


[0.4066093862056732, 0.8102999925613403]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 81.03% 
f1-score is 0.8105841238142786% 
              precision    recall  f1-score   support

           0       0.81      0.81      0.81      5000
           1       0.81      0.81      0.81      5000

    accuracy                           0.81     10000
   macro avg       0.81      0.81      0.81     10000
weighted avg       0.81      0.81      0.81     10000



# Save model

In [ ]:
model.save('FR_lmst_fifth')

INFO:tensorflow:Assets written to: FR_lmst_fifth/assets


In [ ]:
!zip -r /content/Frfifth.zip /content/FR_lmst_fifth/

  adding: content/FR_lmst_fifth/ (stored 0%)
  adding: content/FR_lmst_fifth/keras_metadata.pb (deflated 90%)
  adding: content/FR_lmst_fifth/assets/ (stored 0%)
  adding: content/FR_lmst_fifth/variables/ (stored 0%)
  adding: content/FR_lmst_fifth/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: content/FR_lmst_fifth/variables/variables.index (deflated 66%)
  adding: content/FR_lmst_fifth/saved_model.pb (deflated 79%)
